# Neural Shape Model Tutorial #1

## Learning to Fit Signed Distance Functions (SDFs) with Neural Networks

This tutorial demonstrates what a signed distance field is, and how to use 
a Multi-Layer Perceptron (MLP) neural network to learn a signed distance
function (SDF) of a 3D surface. We'll work with 
medical imaging data (a tibia bone) and train a network to predict 
Signed Distance Function (SDF) values for any coordinate in space. 
These are commonly called "implicit representations" as they implicitly, 
instead of explicitly, represent the surface. E.g., a triangle mesh
explicitly represents the surface, whereas the function doesn't explicitly represent
the surface, but it can be queried to get the surface, therefore it implicitly
represents the surface. These can also be called "coordinate" network as
the input is a coordinate in space. 

### What is a Signed Distance Function (SDF)?

An SDF is a function that, given a 3D point (coordinate) in space, returns:
- **Negative values** for points inside the surface
- **Zero** for points exactly on the surface  
- **Positive values** for points outside the surface

The absolute value represents the distance to the object surface.

### Why Use Neural Networks for Implicit Surfaces?

Traditional explicit representations (like meshes) have limitations:
- Fixed topology
- Memory scales with surface complexity
- Difficult to edit or deform

Neural implicit representations offer:
- **Continuous surfaces** at arbitrary resolution
- **Compact representation** (just network weights)
- **Easy interpolation** between shapes
- **Differentiable** for optimization

### 📚 Papers of Interest
- https://arxiv.org/abs/1901.05103
- https://openreview.net/pdf?id=UuHtdwRXkzw
- https://www.medrxiv.org/content/10.1101/2024.05.06.24306965v2
    - https://ieeexplore.ieee.org/abstract/document/10735783

Let's start by importing the necessary libraries:


In [20]:

# Install required packages if running in Colab
import sys
import os

if 'google.colab' in sys.modules:
    !pip install --upgrade pip setuptools wheel

    # Download requirements.txt from the repo if not present
    if not os.path.exists('requirements_colab.txt'):
        !wget https://raw.githubusercontent.com/gattia/ISB-2025-Shape-Modeling/main/requirements_colab.txt
    # Install requirements
    !pip install -r requirements_colab.txt


In [21]:
# Core libraries for mesh processing and visualization
import pymskt as mskt          # Medical imaging toolkit for mesh operations
import glob                    # File path pattern matching
import os                      # Operating system interface
from itkwidgets import view    # Interactive 3D visualization in Jupyter
import pyvista as pv           # 3D data processing and visualization
import numpy as np             # Numerical computing
import sys

# Plotting library
import matplotlib.pyplot as plt

# PyTorch libraries for deep learning
import torch                   # Main PyTorch library
import torch.nn as nn          # Neural network modules
from torch.utils.data import Dataset, DataLoader  # Data handling utilities

if 'google.colab' in sys.modules:
    from google.colab import output
    output.enable_custom_widget_manager()

In [22]:
# =============================================================================
# DEVICE CONFIGURATION: Set up GPU/CPU device for PyTorch
# =============================================================================

# Automatically detect the best available device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Or manually specify:
# DEVICE = torch.device('cpu')        # Force CPU
# DEVICE = torch.device('cuda')       # Force CUDA/GPU
# DEVICE = torch.device('mps')        # Force Apple Silicon GPU

print(f"Using device: {DEVICE}")
if DEVICE.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory allocated: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")

Using device: cpu


## Step 1: Loading and Visualizing the Reference Mesh

We'll start by loading a 3D mesh of a tibia bone from medical imaging data. This mesh 
represents the explicit surface that we want to learn implicitly using our neural network.

The mesh is stored in VTK format and contains:
- **Vertices**: 3D points defining the surface
- **Faces**: Triangular connectivity between vertices
- **Surface normals**: Vectors perpendicular to the surface


In [23]:
# Load the reference tibia mesh from a VTK file
# This is a 3D triangulated surface mesh of a tibia bone from medical imaging
if 'google.colab' in sys.modules:
    # Download the mesh file from GitHub if running in Colab
    tibia_url = "https://github.com/gattia/ISB-2025-Shape-Modeling/raw/main/data/9226874_RIGHT_tibia.vtk"
    local_mesh_path = "9226874_RIGHT_tibia.vtk"
    if not os.path.exists(local_mesh_path):
        !wget -O {local_mesh_path} {tibia_url}
    path_ref_tibia = local_mesh_path
else:
    path_ref_tibia = './data/9226874_RIGHT_tibia.vtk'
ref_tibia = mskt.mesh.Mesh(path_ref_tibia)

# Visualize the original mesh in 3D
# This shows the explicit surface representation we want to learn implicitly
view(geometries=[ref_tibia])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## Step 2: Data Preparation and Normalization

Before training our neural network, we need to prepare high-quality training data. This step is crucial for successful SDF learning!

### 🎯 Our Data Pipeline

1. **🔧 Normalize the mesh**: Center at origin and scale for numerical stability
2. **✂️ Create 2D slice**: Simplify 3D→2D for this tutorial
3. **🎲 Sample training points**: Generate diverse coordinate samples
4. **📏 Compute ground truth SDFs**: Calculate true signed distances
5. **📊 Visualize the data**: Inspect our training examples

### Why Normalize?

**🔢 Numerical Stability**: 
- Neural networks train best with inputs in consistent ranges (e.g., [-1, 1])
- Large coordinate values can cause gradient instability and slow convergence
- Normalization ensures all features have similar scales

**📐 Scale Consistency**:
- Makes the problem scale-invariant
- Network learns relative distances, not absolute measurements
- Easier to transfer knowledge between different sized objects

### 🎯 Smart Training Data Strategy

**The Challenge**: We need point-SDF pairs to train our network, but we only have a surface mesh.

**Our Solution**: Sample points around the surface using strategic noise injection:

```
1. Start with surface points → Known SDF = 0
2. Add small noise → Points near surface with small |SDF|  
3. Add larger noise → Points far from surface with large |SDF|
4. Add uniform samples → Better coverage of the spatial domain
```

**📊 Three-Part Sampling Strategy:**

1. **🎯 Close points** (σ = 0.01): Dense sampling near surface
   - Ensures accurate surface boundary learning
   - High density where precision matters most

2. **🌍 Far points** (σ = 0.075): Broader spatial coverage  
   - Teaches inside/outside classification
   - Provides context about distance gradients

3. **🎲 Uniform points**: Random spatial coverage
   - Prevents overfitting to surface-centric distribution
   - Improves generalization across the domain

**💡 Why This Works:**
- **Surface fidelity**: Dense near-surface samples capture fine details
- **Spatial coverage**: Broader sampling teaches global SDF structure  
- **Balanced learning**: Network learns both precise distances AND correct signs


In [24]:
# =============================================================================
# MESH NORMALIZATION: Prepare the mesh for neural network training
# =============================================================================

# Step 1: Center the mesh at origin
mean = np.mean(ref_tibia.points, axis=0)
ref_tibia.points -= mean

# Step 2: Scale to unit norm  
# This normalizes the scale and improves numerical stability during training
norm = np.linalg.norm(ref_tibia.points, axis=1)  # Distance from origin for each point
max_norm = np.max(norm)                          # Maximum distance
ref_tibia.points /= max_norm                     # Scale all points to be within unit sphere (with buffer)

# =============================================================================
# TRAINING DATA GENERATION: Create randomly sampled points & compute SD values
# =============================================================================

# Create a 2D coronal slice (cutting through y-axis) to simplify the problem
# This reduces the complexity from 3D to 2D for our example. 
slice_ = ref_tibia.slice('y', origin=(0, 0, 0))

# Training parameters
N = 20_000          # Number of points per distribution
CLOSE_SD = 0.01     # Small standard deviation for points near surface
FAR_SD = 0.075      # Larger standard deviation for points away from surface

# Initialize array to hold all training points:
# 2 distributions × N points each + 500 extra points to get uniformly over the space. 
pts = np.zeros((N*2 + 500, 3))

# Generate two different point distributions around the surface
for i, SD in enumerate([CLOSE_SD, FAR_SD]):
    # Randomly sample points on the slice surface as starting locations
    indices = (np.random.sample(N) * slice_.points.shape[0]).astype(int)
    
    # Generate Gaussian noise to perturb the surface points
    # This creates points both inside (negative SDF) and outside (positive SDF) the surface
    x_noise = np.random.normal(loc=0, scale=SD, size=N)
    z_noise = np.random.normal(loc=0, scale=SD, size=N)
    
    # Add noise to the x and z coordinates (y stays at slice level ≈ 0)
    pts[i*N:(i+1)*N, 0] = slice_.points[indices, 0] + x_noise  # X coordinate
    pts[i*N:(i+1)*N, 2] = slice_.points[indices, 2] + z_noise  # Z coordinate
    # Y coordinate remains 0 (slice level)

# get random uniform points over the unite cube to make the 
# network learn the whole space.
x_uniform = (np.random.rand(500)*2 - 1)   # make points span -1, 1
z_uniform = (np.random.rand(500)*2 - 1)   # make points span -1, 1

pts[-500:, 0] = x_uniform
pts[-500:, 2] = z_uniform


# =============================================================================
# SDF COMPUTATION: Calculate ground truth signed distances
# =============================================================================

# Convert points to PyVista format for SDF computation
pts_ = pv.PolyData(pts)

# Compute the signed distance from each point to the mesh surface
# This gives us the ground truth SDF values that our network will learn to predict
pts_.compute_implicit_distance(ref_tibia, inplace=True)

# =============================================================================
# VISUALIZATION: Display the generated training data
# =============================================================================

# Visualize the slice, original mesh, and generated training points
# Points are colored by their SDF values
view(geometries=[slice_, ref_tibia], point_sets=pts_, point_size=2)


Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## Step 3: Neural Network Architecture and Training

Now we'll design and train a neural network to learn the implicit surface representation. You can think of a neural network as a "function". Thus, after training the network to predict the signed
distance it is a continuous "signed distance function".

### Network Architecture: Multi-Layer Perceptron (MLP)

Our network is a simple but powerful feedforward neural network:

# **Network Architecture Overview**
# 
# | Layer            | Input Size      | Output Size     | Description                |
# |------------------|----------------|-----------------|----------------------------|
# | Input            | (x, z)         | 2               | 2D coordinates             |
# | Hidden Layer 1   | 2              | 32              | 32 neurons, ReLU           |
# | Hidden Layer 2   | 32             | 32              | 32 neurons, ReLU           |
# | Output           | 32             | 1               | SDF value (scalar)         |
# 
# **Diagram:**
# 
# (x, z) → [Hidden 1: 32] → [Hidden 2: 32] → [Output: 1 SDF value]

**The network IS the SDF function**: Once trained, `model(x, z)` returns the signed distance to the surface at coordinate (x, z).

### Key Design Choices:

**🎯 Why These Architecture Decisions?**

1. **Input dimension**: 2D (x,z) instead of 3D simplifies learning while demonstrating core concepts
2. **Hidden size**: 32 neurons provide sufficient capacity without overfitting on our 40K samples
3. **Activation**: ReLU creates non-linear decision boundaries essential for complex shapes
4. **Output**: No final activation → continuous real-valued SDF predictions
5. **Depth**: 2 hidden layers balance expressiveness with training stability

**💡 Experiment Ideas:**
- Try different activations: `Tanh`, `Sigmoid`, `Swish` - how do they affect surface smoothness?
    or learning convergence, etc. 
- Modify architecture: More layers? Wider networks? Skip connections?
- Add positional encoding for better high-frequency detail capture?
    - Convert 2D input into a higher dimensional representation of position. 

### Training Strategy & Loss Function Design

**🎯 L1 Loss (Mean Absolute Error)**
- **Why L1?** Preserves sharp surface features better than L2/MSE
- **L2 tends to blur** → smooth but less accurate surfaces  
- **L1 preserves detail** → sharper, more accurate reconstructions

**🚀 Advanced Loss Ideas to Try:**
- **Weighted L1**: Higher weights for points near surface (|SDF| < threshold)
- **Sign-aware loss**: Extra penalty for wrong inside/outside classification
- **Eikonal loss**: Regularize SDF gradients to have unit magnitude (proper distance field property)

**⚙️ Training Configuration:**
- **Optimizer**: Adam with adaptive learning rates (robust for coordinate networks)
- **Batch size**: 500 points for good gradient estimates without memory issues
- **SDF clipping**: Focus learning on surface proximity (±0.1 units)
- **Data shuffling**: Prevents overfitting to specific point sequences

**📊 What to Watch During Training:**
- **Decreasing loss**: Network learning the coordinate→SDF mapping
- **Convergence speed**: Should reach <0.01 loss within 100-200 epochs
- **Loss plateaus**: May indicate need for learning rate adjustment or architecture changes 
    


In [25]:
# =============================================================================
# NEURAL NETWORK ARCHITECTURE: Define the MLP for SDF prediction
# =============================================================================

class SimpleMLP(nn.Module):
    """
    A Multi-Layer Perceptron for learning coordinate-to-SDF mappings.
    
    Architecture:
    - Input: 2D coordinates (x, z)
    - Two hidden layers with ReLU activations
    - Output: Single SDF value
    """
    def __init__(self, input_dim=2, hidden_dim=64, output_dim=1):
        super(SimpleMLP, self).__init__()
        
        # Define the network as a sequence of layers
        self.net = nn.Sequential(
            # First hidden layer: projects 2D input to hidden dimension
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),  # Non-linear activation
            
            # Second hidden layer: processes features
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),  # Non-linear activation
            
            # Output layer: produces single SDF value
            nn.Linear(hidden_dim, output_dim)
            # No activation - we want continuous real-valued output
        )

    def forward(self, x):
        """Forward pass: coordinate -> SDF prediction"""
        return self.net(x)

# =============================================================================
# DATASET CLASS: Prepare training data for PyTorch
# =============================================================================

class PointsSDFDataset(Dataset):
    """
    PyTorch Dataset for SDF learning.
    
    Features:
    - Extracts 2D coordinates (x, z) from 3D points
    - Clips SDF values to focus learning on surface region
    - Handles batching and shuffling for training
    """
    def __init__(self, points, sdf, max_sdf=0.1):
        """
        Args:
            points: numpy array of shape (N, 3) - 3D coordinates
            sdf: numpy array of shape (N,) - ground truth SDF values
            max_sdf: maximum SDF value to consider (clips outliers)
        """
        # Extract only x and z coordinates (ignore y since we're working with a slice)
        self.xz = torch.tensor(points[:, [0, 2]], dtype=torch.float32).to(DEVICE)
        
        # Convert SDF values to tensor and add batch dimension
        self.sdf = torch.tensor(sdf, dtype=torch.float32).unsqueeze(1).to(DEVICE)
        
        # Clip SDF values to focus learning on surface proximity
        # This prevents the network from spending effort learning exact signed distance
        # values for points far from surface... higher accuracy there doesn't help us. 
        self.sdf = torch.clamp(self.sdf, min=-max_sdf, max=max_sdf)

    def __len__(self):
        """Return number of training samples"""
        return self.xz.shape[0]

    def __getitem__(self, idx):
        """Return a single training example: (coordinates, sdf_value)"""
        return self.xz[idx], self.sdf[idx]

# =============================================================================
# TRAINING SETUP: Initialize model, data, and optimization components
# =============================================================================

# Training parameters
num_epochs = 500
batch_size = 500
network_hidden_dimension = 32

# Create dataset and dataloader for efficient batch processing
dataset = PointsSDFDataset(pts, pts_['implicit_distance'])
dataloader = DataLoader(
    dataset, 
    batch_size=batch_size,     # Process 500 points at once for efficiency
    shuffle=True        # Randomize order each epoch to prevent overfitting
)

# Initialize the neural network
model = SimpleMLP(input_dim=2, hidden_dim=network_hidden_dimension, output_dim=1)

# Move model to the specified device
model = model.to(DEVICE)

# Setup optimization
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  # setup optimizer... try different learning rates?
criterion = nn.L1Loss()  # Mean Absolute Error - robust to outliers, less blurring. 



print(f"Starting training for {num_epochs} epochs...")
print(f"Training data: {len(dataset)} points")
print(f"Batch size: {dataloader.batch_size}")
print(f"Network parameters: {sum(p.numel() for p in model.parameters())}")

# =============================================================================
# TRAINING LOOP: Learn the coordinate-to-SDF mapping
# =============================================================================

for epoch in range(num_epochs):
    running_loss = 0.0
    
    # Process all batches in the dataset
    for batch_idx, (xz_batch, sdf_batch) in enumerate(dataloader):
        # Clear gradients from previous iteration
        optimizer.zero_grad()
        
        # Forward pass: predict SDF values for batch of coordinates
        sdf_pred = model(xz_batch)
        
        # Compute loss / error we want to optimize. 
        loss = criterion(sdf_pred, sdf_batch)
        
        # Backward pass: compute gradients via backpropagation
        loss.backward()
        
        # Update model parameters using computed gradients
        optimizer.step()
        
        # Accumulate loss for epoch statistics
        running_loss += loss.item() * xz_batch.size(0)
    
    # Calculate average loss for this epoch
    epoch_loss = running_loss / len(dataset)
    
    # Log progress every 20 epochs
    if epoch % 20 == 0:
        print(f"Epoch {epoch+1:3d}/{num_epochs}, Loss: {epoch_loss:.6f}")



Starting training for 500 epochs...
Training data: 40500 points
Batch size: 500
Network parameters: 1185
Epoch   1/500, Loss: 0.034792
Epoch  21/500, Loss: 0.014379
Epoch  41/500, Loss: 0.008977
Epoch  61/500, Loss: 0.007082
Epoch  81/500, Loss: 0.006420
Epoch 101/500, Loss: 0.005904
Epoch 121/500, Loss: 0.005427
Epoch 141/500, Loss: 0.005170
Epoch 161/500, Loss: 0.004895
Epoch 181/500, Loss: 0.004724
Epoch 201/500, Loss: 0.004609
Epoch 221/500, Loss: 0.004531
Epoch 241/500, Loss: 0.004443
Epoch 261/500, Loss: 0.004391
Epoch 281/500, Loss: 0.004303
Epoch 301/500, Loss: 0.004244
Epoch 321/500, Loss: 0.004196
Epoch 341/500, Loss: 0.004157
Epoch 361/500, Loss: 0.004084
Epoch 381/500, Loss: 0.004053
Epoch 401/500, Loss: 0.003974
Epoch 421/500, Loss: 0.003943
Epoch 441/500, Loss: 0.003892
Epoch 461/500, Loss: 0.003827
Epoch 481/500, Loss: 0.003805


## Step 4: Evaluating the Learned SDF

Our network should now be trained to map coordinates to signed distance values. 
The decreasing loss indicates that the network is getting better at predicting 
the correct SDF values.

Now let's evaluate how well our network learned the implicit surface representation:

### What we'll do:
1. **Create a dense grid** of query points across our 2D slice
    - This could be thought of/can be used to re-create an image.
    - However, we do not need to query on an explicit grid. We can query anywhere in space. 
2. **Predict SDF values** for each grid point using our trained network
3. **Visualize the learned SDF field** as colored points
4. **Extract the surface** using marching cubes on the zero level 
    set (where SD crosses from (-) to (+))
5. **Compare** with the original surface

### Understanding the Results:
- **Negative SDF values**: Points inside the bone
- **Positive SDF values**: Points outside the bone  
- **Values near zero**: Points close to the surface boundary
- **Zero level set**: The reconstructed surface where SDF = 0

The quality of our implicit representation depends on how well the network 
learned these relationships!


### Visualization tips: 
- The default colormap is viridis (purple to yellow). 
- The colormap that is blue to red with white in the middle does well for visualizing the SDF field.
- Look at the range of values, often it is not symmetric. It helps to make it the min/max be -0.1 to 0.1 to help visualization the colormap. 


# **Note:**  
The network size and training parameters were chosen for efficient model fitting in this tutorial. The reconstructed surfaces are not 
exact and can be improved. After running the tutorial, try experimenting with different parameters to optimize the results.


In [26]:
# =============================================================================
# SDF FIELD VISUALIZATION: Evaluate the learned implicit representation
# =============================================================================

print("Generating dense grid for SDF evaluation...")

# Create a high-resolution grid of query points across our 2D slice
GRID_RESOLUTION = 0.01  # Distance between grid points (smaller = higher resolution)
x = np.arange(-1, 1.01, GRID_RESOLUTION)
z = np.arange(-1, 1.01, GRID_RESOLUTION)

print(f"Grid size: {len(x)} × {len(z)} = {len(x) * len(z):,} query points")

# Create coordinate meshes and flatten for network input
xx, zz = np.meshgrid(x, z)
xz_grid = np.stack([xx.ravel(), zz.ravel()], axis=1)

# Convert to PyTorch tensor for model inference
xz_tensor = torch.from_numpy(xz_grid).float().to(DEVICE)

print("Predicting SDF values using trained network...")

# Get SDF predictions from our trained network
# No gradients needed since we're just evaluating (not training)
with torch.no_grad():
    sdf_pred = model(xz_tensor).cpu().numpy().flatten()

print(f"SDF predictions range: [{sdf_pred.min():.4f}, {sdf_pred.max():.4f}]")
print(f"Points inside surface (SDF < 0): {(sdf_pred < 0).sum():,}")
print(f"Points outside surface (SDF > 0): {(sdf_pred > 0).sum():,}")

# =============================================================================
# PREPARE VISUALIZATION DATA: Convert predictions to 3D points for display
# =============================================================================

# Combine coordinates into 3D points (x, 0, z) - y=0 since we're working with a slice
xyz_coords = np.stack([
    xz_grid[:, 0],                    # X coordinates
    np.zeros_like(xz_grid[:, 0]),     # Y = 0 (slice level)
    xz_grid[:, 1]                     # Z coordinates
], axis=1)

# Create PyVista point cloud for visualization
xyz_points = pv.PolyData(xyz_coords)

# Assign predicted SDF values as scalar data for color mapping
xyz_points['Predicted_SDF'] = sdf_pred

print("\nVisualizing learned SDF field...")
print("If using suggesting visualization changes from above...")
print("Color mapping: Blue = Inside surface (negative SDF), Red = Outside surface (positive SDF)")

# Visualize the learned SDF field alongside the original slice
view(geometries=[slice_], point_sets=xyz_points, point_size=2)


Generating dense grid for SDF evaluation...
Grid size: 201 × 201 = 40,401 query points
Predicting SDF values using trained network...
SDF predictions range: [-0.2956, 0.1475]
Points inside surface (SDF < 0): 16,215
Points outside surface (SDF > 0): 24,186

Visualizing learned SDF field...
If using suggesting visualization changes from above...
Color mapping: Blue = Inside surface (negative SDF), Red = Outside surface (positive SDF)


Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## Step 5: Surface Reconstruction with Marching Cubes

The previous visualization showed us the raw SDF field as colored points. Now we'll 
extract the actual surface by finding where the SDF crosses zero using marching cubes.

### Marching Cubes Algorithm

Marching cubes is a classic algorithm that:
1. **Takes a 3D grid** of scalar values (our SDF predictions)
2. **Finds the zero level set** (where SDF transitions from negative to positive)
3. **Generates triangulated mesh** that represents the explicit surface

### Why the Zero Level Set?

Remember our SDF definition:
- **SDF < 0**: Inside the surface  
- **SDF = 0**: Exactly on the surface ← This is what we want!
- **SDF > 0**: Outside the surface

The zero level set gives us the reconstructed surface from our learned implicit representation.

### Implementation Notes:

- We create a structured 3D grid for marching cubes
- Our slice has very small Y dimension (thickness ≈ 0.02) 
- The algorithm will find where our network predicts SDF ≈ 0


In [27]:
# Visualize the predictions of the model... 

# Create a grid of x and z from -1 to 1 in steps of 0.01
x = np.arange(-1, 1.01, 0.01)
z = np.arange(-1, 1.01, 0.01)
xx, zz = np.meshgrid(x, z)
xz_grid = np.stack([xx.ravel(), zz.ravel()], axis=1)
xz_tensor = torch.from_numpy(xz_grid).float().to(DEVICE)

# Get SDF predictions from the trained model
with torch.no_grad():
    sdf_pred = model(xz_tensor).cpu().numpy().flatten()

# combing the predictions into 3D points (x, 0, z)
xyz = np.stack([xz_grid[:, 0], np.zeros_like(xz_grid[:, 0]), xz_grid[:, 1]], axis=1)

# creeate polydata from points for visualization. 
xyz = pv.PolyData(xyz)

# assign sdf to xyz for visualization.
xyz['sdf'] = sdf_pred

view(geometries=[slice_], point_sets=xyz)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [28]:
# =============================================================================
# SURFACE RECONSTRUCTION: Extract implicit surface using marching cubes
# =============================================================================

print("Setting up 3D grid for marching cubes...")

# Create a structured 3D grid for the marching cubes algorithm
MARCHING_CUBES_RESOLUTION = 100  # Grid resolution (higher = more detail, slower)

# Define the spatial bounds for our reconstruction
x_min, y_min, z_min = -1.0, -0.01, -1.0   # Minimum coordinates
x_max, y_max, z_max =  1.0,  0.01,  1.0    # Maximum coordinates

# Calculate grid spacing based on resolution
spacing = (
    (x_max - x_min) / (MARCHING_CUBES_RESOLUTION - 1),  # X spacing
    (y_max - y_min) / 1,                                # Y spacing (thin slice)
    (z_max - z_min) / (MARCHING_CUBES_RESOLUTION - 1),  # Z spacing
)

# Create the structured grid using PyVista
grid = pv.ImageData(
    dimensions=(MARCHING_CUBES_RESOLUTION, 2, MARCHING_CUBES_RESOLUTION),
    spacing=spacing,
    origin=(x_min, y_min, z_min),
)

# Extract grid coordinates for SDF evaluation
x, y, z = grid.points.T


# =============================================================================
# SDF EVALUATION: Predict signed distances for the marching cubes grid
# =============================================================================

print("Evaluating SDF at grid points...")

# Prepare input for our neural network: extract (x, z) coordinates
xz_grid_3d = np.stack([x, z], axis=1)
xz_tensor_3d = torch.from_numpy(xz_grid_3d).float().to(DEVICE)

# Predict SDF values for all grid points using our trained model
with torch.no_grad():
    sdf_pred_3d = model(xz_tensor_3d).cpu().numpy().flatten()

print(f"Grid SDF range: [{sdf_pred_3d.min():.4f}, {sdf_pred_3d.max():.4f}]")

# =============================================================================
# MARCHING CUBES: Extract the zero level set as a triangulated surface
# =============================================================================

print("Running marching cubes to extract surface...")

# Extract the isosurface where SDF = 0 (the learned surface boundary)
reconstructed_mesh = grid.contour([0], sdf_pred_3d, method='marching_cubes')

# Compute surface normals for proper shading and visualization
reconstructed_mesh.compute_normals(inplace=True)


# =============================================================================
# VISUALIZATION: Compare reconstructed surface with original
# =============================================================================

# Display both the reconstructed surface and original slice for comparison
view(geometries=[reconstructed_mesh, slice_])

Setting up 3D grid for marching cubes...
Evaluating SDF at grid points...
Grid SDF range: [-0.2936, 0.1465]
Running marching cubes to extract surface...


Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## 🎉 Tutorial Complete: What We Accomplished

Congratulations! You've successfully implemented a neural implicit surface representation using 
signed distance functions. Here's what we achieved:

### Key Accomplishments

1. **📊 Learned SDF Theory**: Understood how signed distance functions represent surfaces implicitly
2. **🔢 Generated Training Data**: Created noise-perturbed surface samples with ground truth SDF values
3. **🧠 Built Neural Network**: Designed an MLP to map coordinates → signed distances
4. **📈 Trained Successfully**: Achieved low reconstruction error
5. **🎨 Visualized Results**: Saw the learned SDF field as colored point clouds
6. **🔺 Reconstructed Surface**: Used marching cubes to extract the zero level set

### What Makes This Powerful?

- **Continuous Representation**: Query SDF at any coordinate, not just discrete mesh vertices
- **Compact Storage**: Entire surface encoded in neural network weights
- **Differentiable**: Can optimize, interpolate, and manipulate using gradients
- **Resolution Independent**: Reconstruct at any desired level of detail

### Next Steps & Extensions

Want to explore further? Try these modifications:

**🎛️ Experiment with Architecture:**
- Different activation functions (Tanh, Sigmoid, Swish)
- More/fewer hidden layers
- Positional encoding for high-frequency details

**📊 Improve Training:**
- Different loss functions (L2, Huber, custom SDF losses)
- Better sampling strategies (progressive, importance sampling)
- Regularization techniques

**🌍 Scale to 3D:**
- Full 3D coordinate inputs (x, y, z)
- Multiple bone shapes for generalization
- Conditional networks for shape families

**⚡ Advanced Applications:**
- Shape interpolation between different bones
- Deformation and editing operations
- Integration with physics simulations

This tutorial provided the foundation for neural implicit 
representations – a rapidly growing field with applications 
in computer graphics, medical imaging, and 3D AI!
